## Match Index Notebook

This Notebook is for index functions

### Load Packages

In [1]:
import pandas as pd
import os
import re

### Set Working Directory

In [2]:
parent_dir = os.path.abspath('../..') # ".." means go back 1 directory!
file_path = os.path.join(parent_dir, 'data/mens/mens_results.csv')

### Read Data

In [ ]:
matches = pd.read_csv(file_path)
matches.head()

# Update: Only Read in Rows from this Season! (subset starting from September ? 2024)

,Event Name,Date,Player1,Player1 UTR,Player2,Player2 UTR,Score
0,"Dual Match: University of California, Los Ange...",2025-04-26,Emon Van Loben Sels,13.59,Alexander Bernard,13.33,"7-6(5), 5-7, 6-3"
1,"Dual Match: University of California, Los Ange...",2025-04-26,Alexander Hoogmartens,13.00,Jack Anthrop,13.66,"7-5, 0-6, 6-4"
2,"Dual Match: University of California, Los Ange...",2025-04-26,Aidan Kim,13.75,Rudy Quan,13.67,"7-6(3), 6-2"
3,"Dual Match: University of California, Los Ange...",2025-04-26,Preston Stearns,13.08,Aadarsh Tripathi,12.94,"6-3, 2-6, 6-2"
4,"Dual Match: University of California, Los Ange...",2025-04-26,Kaylan Bigun,13.17,William Jansen,13.24,"6-2, 7-5"


### Set Player to Query

In [4]:
# Set Player Name!
player_name = 'Rudy Quan'
school = "University of California, Los Angeles"
# date = ?

### index_csv() Function

#### Helper Function to Flip Scores

In [5]:
# Helper function to flip scores like "7-6(5), 5-7, 6-3"
def flip_score(score_str):
    if not isinstance(score_str, str):
        return score_str
    flipped = []
    for set_score in score_str.split(', '):
        match = re.match(r'(\d+)-(\d+)(\(\d+\))?', set_score)
        if match:
            p1, p2, tiebreak = match.groups()
            flipped_score = f"{p2}-{p1}"
            if tiebreak:
                flipped_score += tiebreak
            flipped.append(flipped_score)
        else:
            flipped.append(set_score)  # Preserve unrecognized formats
    return ', '.join(flipped)


# UPDATE do not include the comma in final result

In [ ]:
def index_csv(data, player_name, school):

    # Filter for Player
    df = data[(data['Player1'] == player_name) | (matches['Player2'] == player_name)].copy()

    
    # Extract opponent name from 'Event Name'
    def get_opponent(event):
        if not isinstance(event, str):
            return None
        teams = re.findall(r'vs\s(.+)', event)
        if school in event and teams:
            return teams[0].strip() if school in event.split('vs')[0] else event.split('vs')[0].replace('Dual Match:', '').strip()
        return None

    # Create opponent_school Column
    df['opponent_school'] = df['Event Name'].apply(get_opponent)

    # Create opponent Column
    df['opponent'] = df.apply(lambda row: row['Player2'] if row['Player1'] in player_name 
                                    else (row['Player1'] if row['Player2'] in player_name 
                                    else None
                                    ), axis=1
                                    )
    
    # UPDATE: Make all names to where the only the first letter of each name is capitalized
    # eg. Aristotelis THANOS --> Aristotelis Thanos

    # Create ucla_result Column
    df['ucla_result'] = df.apply(lambda row: 'Win' if row['Player1'] in player_name 
                                            else ('Loss' if row['Player2'] in player_name 
                                            else None), axis=1
                                            )
    
    # UPDATE: Make Win and Loss --> "W" and "L" AND take into account UNFINISHED matches "UF"
    # UPDATE: Combine result and score column to say result after score
    # eg. 6-3 6-3 W  | 3-6 4-6 L  | 2-6 6-5 UF
    
    # Create column Score (UCLA Perspective)
    df['Score (UCLA Perspective)'] = df.apply(lambda row: flip_score(row['Score']) 
                                                        if row['Player2'] in player_name 
                                                        else row['Score'],
                                                        axis=1
                                                        )
    
    # UPDATE: make date column to mm/dd/yy format
    # eg. 2025-04-26 --> 04/26/25

    
    final_df = df[['opponent_school', 
                   'opponent', 
                   'Score (UCLA Perspective)', 
                   'Date']].rename(columns={'Score (UCLA Perspective)': 'score',
                                            'Date': 'date'
                                            })

    return final_df

### Output Function Result

In [7]:
index_csv(matches, player_name, school)

# UPDATE: Output to csv in index_page folder

,opponent_school,opponent,score,date
2,The Ohio State University,Aidan Kim,"6-7(3), 2-6",2025-04-26
10,Michigan State University,Aristotelis THANOS,"4-6, 3-1",2025-04-25
17,University of Michigan,William Cooksey,"6-3, 5-6",2025-04-24
23,University of Nebraska-Lincoln,Calvin MUELLER,"3-6, 6-3, 6-1",2025-04-19
25,University of Wisconsin-Madison,Michael Minasyan,"6-2, 3-6",2025-04-17
...,...,...,...,...
4790,None,Matthew Trinidad,"6-0, 6-0",2014-05-10
4804,None,Kyle Kang,"0-6, 1-6",2014-04-12
4805,None,Prayag Ahire,"6-2, 6-0",2014-04-12
4810,None,Heaton Manor,"6-0, 6-0",2014-04-11


### Old Niranjan + Emma Work

In [ ]:
# Set UCLA name
my_school = "University of California, Los Angeles"

# Extract opponent name from 'Event Name'
def get_opponent(event):
    if not isinstance(event, str):
        return None
    teams = re.findall(r'vs\s(.+)', event)
    if my_school in event and teams:
        return teams[0].strip() if my_school in event.split('vs')[0] else event.split('vs')[0].replace('Dual Match:', '').strip()
    return None

matches['opponent_school'] = matches['Event Name'].apply(get_opponent)

# Set of UCLA players
ucla_players = {
    "Andrei Crabel", "Alexander Hoogmartens", "Spencer Johnson", "Rudy Quan",
    "Giacomo Revelli", "Aadarsh Tripathi", "Gianluca Ballotta", "Kaylan Bigun",
    "Cassius Chinlund", "Emon van Loben Sels", "Leo von Bismarck"
}

matches['opponent'] = matches.apply(
    lambda row: row['Player2'] if row['Player1'] in ucla_players else (
        row['Player1'] if row['Player2'] in ucla_players else None
    ),
    axis=1
)

matches['ucla_result'] = matches.apply(
    lambda row: 'Win' if row['Player1'] in ucla_players else (
        'Loss' if row['Player2'] in ucla_players else None
    ),
    axis=1
)

# Helper function to flip scores like "7-6(5), 5-7, 6-3"
def flip_score(score_str):
    if not isinstance(score_str, str):
        return score_str
    flipped = []
    for set_score in score_str.split(', '):
        match = re.match(r'(\d+)-(\d+)(\(\d+\))?', set_score)
        if match:
            p1, p2, tiebreak = match.groups()
            flipped_score = f"{p2}-{p1}"
            if tiebreak:
                flipped_score += tiebreak
            flipped.append(flipped_score)
        else:
            flipped.append(set_score)  # Preserve unrecognized formats
    return ', '.join(flipped)

# Flip the score only if Player 2 is a UCLA player
matches['Score (UCLA Perspective)'] = matches.apply(
    lambda row: flip_score(row['Score']) if row['Player2'] in ucla_players else row['Score'],
    axis=1
)

matches

# filter through each player on ucla team
## Gianluca Ballotta
gianlucaBallotta_df = matches[(matches['Player1'] == 'Gianluca Ballotta') | (matches['Player2'] == 'Gianluca Ballotta')]
gianlucaBallotta_df
## Kaylan Bigun
kaylanBigun_df = matches[(matches['Player1'] == 'Kaylan Bigun') | (matches['Player2'] == 'Kaylan Bigun')]
kaylanBigun_df
## Cassius Chinlund
cassius_df = matches[(matches['Player1'] == 'Kaylan Bigun') | (matches['Player2'] == 'Kaylan Bigun')]
kaylanBigun_df
## Andrei Crabel
## Alexander Hoogmartens
## Spencer Johnson
## Rudy Quan
rudy_df = matches[(matches['Player1'] == 'Rudy Quan') | (matches['Player2'] == 'Rudy Quan')]
rudy_df
## Giacomo Revelli
## Aadarsh Tripathi
## Emon van Loben Sels
## Leo von Bismarck

In [ ]:
final_df = rudy_df[['opponent_school', 'opponent', 'Score (UCLA Perspective)', 'Date']].rename(columns={
    'Score (UCLA Perspective)': 'score',
    'Date': 'date'})

final_df

In [ ]:
# Create a function that returns data in this format
file_path = os.path.join(parent_dir, 'data/practice/index_sample.csv')
sample_output = pd.read_csv(file_path)
sample_output

In [ ]:
# Column Format Guidelines:

# school_name: Full School Name
# opponent: Full Name
# score: Respect to Player we are querying for (switch score if ucla player lost)
# date: 00/00/00